In [1]:
#fas-정재환
import cx_Oracle
import pandas as pd

In [2]:
#오라클 서버  접속
conn = cx_Oracle.connect('scott/c4iuser11@localhost:1521/orcl')
#커서 연결
cursor = conn.cursor()
#건강검진 정보 데이터 조회
sql = "select * from health"

#쿼리문 실행
#판다스에서 제공하는 read_sql함수를 사용하여 질의하면 컬럼명도 얻을 수 있다.
rs = pd.read_sql(sql, conn)
rs

,번호,성별,연령,신장,체중,허리둘레,수축기혈압,이완기혈압,식전혈당,총콜레스테롤,흡연상태,음주여부
0,648,2,60,155,55,82.0,128,80,99,181,1,0
1,649,2,30,155,60,73.0,127,89,78,195,1,0
2,650,1,35,180,80,88.0,131,72,123,148,1,0
3,651,1,30,175,70,73.0,123,64,87,157,3,0
4,652,2,60,160,55,77.0,110,69,82,212,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
199772,199396,1,45,180,95,101.1,112,70,81,206,1,0
199773,199397,1,45,170,60,73.0,110,65,100,194,1,1
199774,199398,1,60,175,50,74.0,118,72,79,194,2,1
199775,199399,1,80,170,75,92.5,121,68,87,186,2,0


In [4]:
del rs

In [18]:
# 체중을 기준으로 그룹화 하고 그룹별 평균 혈당이 전체 혈당 평균값에 5를 더한 값 보다 큰 그룹의 체중과 평균 혈당을 구한다
sql  = 'select 체중, avg(식전혈당) from health group by 체중 having avg(식전혈당) > ' 
sql += '(select avg(식전혈당)+5 from health) order by 체중'
rs = pd.read_sql(sql, conn)
rs

,체중,AVG(식전혈당)
0,90,106.113253
1,95,106.404431
2,100,107.943182
3,105,107.360248
4,110,109.232143
5,115,115.069444
6,125,114.692308
7,130,107.000000
8,135,141.000000


In [25]:
# 체중이 적은 사람, 중간인 사람, 큰 사람의 평균 혈당을 각각 계산한다
sql  = 'select small.blood_sugar, middle.blood_sugar, big.blood_sugar from ' 
sql += '(select avg(식전혈당) blood_sugar from health where 체중 < 65) small, '
sql += '(select avg(식전혈당) blood_sugar from health where 체중 >= 65 and 체중 < 85) middle, '
sql += '(select avg(식전혈당) blood_sugar from health where 체중 >= 85) big'
rs = pd.read_sql(sql, conn)
rs

,BLOOD_SUGAR,BLOOD_SUGAR,BLOOD_SUGAR
0,97.62126,103.035368,106.047645


In [3]:
# 이번에는 행사 개최 정보 공공데이터 테이블을 사용해보자
sql  = 'select count(*) from seminar' 
rs = pd.read_sql(sql, conn)
rs

,COUNT(*)
0,1386


In [7]:
# 2018년 9월1일 이후 개최된 행사의 수
sql  = "select count(*) from seminar where start_date >= to_date('20180901', 'yyyymmdd')" 
rs = pd.read_sql(sql, conn)
rs

,COUNT(*)
0,893


In [17]:
# 2019년도에 개최된 행사를 모두 출력
sql  = "select * from seminar where start_date like '19%'" 
rs = pd.read_sql(sql, conn)
rs

,NUM,FIELD,AREA,TITLE,SOURCE,START_DATE,STOP_DATE,REGISTER_DATE,UPDATE_DATE,READ_COUNT
0,1146,경영,서울특별시,[서울] 2018년 사회적경제 아카데미 갈등관리 교육 안내,서울시사회적경제지원센터,2019-01-01,2019-02-28,2018-12-14,2018-12-14,0
1,1145,경영,울산광역시,[울산] 2018년 제2기분 부가가치세 신고 실무교육,대한상공회의소,2019-01-15,2019-01-15,2018-12-13,2018-12-13,0
2,1133,경영,울산광역시,[울산] 2019년 신고대비 법인결산과 세무조정ㆍ신고 실무교육 안내,울산상공회의소,2019-01-09,2019-01-09,2018-12-07,2018-12-07,0
3,1123,창업,대구광역시,대구콘텐츠코리아랩 콘텐츠 창업 아카데미(상세페이지 제작 과정) 개최 안내,대구콘텐츠코리아랩,2019-01-04,2019-01-25,2018-12-05,2018-12-05,0
4,1374,인력,서울특별시,산업단지 입주기업 R&D 4월 교육 안내,한국산업단지공단,2019-04-03,2019-04-17,2019-03-29,2019-03-29,3
...,...,...,...,...,...,...,...,...,...,...
223,1154,경영,인천광역시,고용노동부 포괄임금계약 지도지침에 따른 노무관리 실무강좌 개최,대한상공회의소,2019-01-17,2019-01-17,2018-12-18,2018-12-18,0
224,1153,인력,인천광역시,2019년 1월 사무관리분야 교육 안내,대한상공회의소,2019-01-15,2019-01-28,2018-12-18,2018-12-18,0
225,1152,인력,인천광역시,2019년 신고대비 법인결산 및 세무조정신고 실무강좌 안내,대한상공회의소,2019-01-08,2019-01-08,2018-12-18,2018-12-18,0
226,1151,인력,인천광역시,2018년 귀속 연말정산 실무강좌 안내,대한상공회의소,2019-01-03,2019-01-03,2018-12-18,2018-12-18,0
